In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [2]:
def scrape_youtube_comments(video_title, max_comments):
    # Set up Chrome browser options
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Open in full screen
    # Don't run in headless mode so comments load properly

    # Start the browser
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        # Step 1: Search for the video on YouTube
        print(f"\n Searching for: {video_title}")
        query = video_title.replace(" ", "+")
        search_url = f"https://www.youtube.com/results?search_query={query}"
        driver.get(search_url)
        time.sleep(3)

        # Step 2: Click the first video result
        first_result = driver.find_element(By.ID, "video-title")
        video_link = first_result.get_attribute("href")
        print(f" Opening video: {video_link}")
        driver.get(video_link)
        time.sleep(5)

        # Step 3: Scroll to load comments
        print(" Loading comments...")
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # Scroll more to load additional comments
        for _ in range(8):
            driver.execute_script("window.scrollBy(0, 2000);")
            time.sleep(2)

        # Step 4: Get the comments
        comment_boxes = driver.find_elements(By.CSS_SELECTOR, "#content-text")
        comments = []

        for comment in comment_boxes:
            text = comment.text.strip()
            if text:
                comments.append(text)
            if len(comments) >= max_comments:
                break

        # Step 5: Save to CSV
        if comments:
            df = pd.DataFrame({"Comment": comments})
            df.to_csv("youtube_comments.csv", index=False)
            print(f" Saved {len(comments)} comments to 'youtube_comments.csv'")
        else:
            print(" No comments found.")

    except Exception as e:
        print(f" Error: {e}")

    finally:
        driver.quit()


In [3]:
# Run the script
if __name__ == "__main__":
    video_title = input(" Enter YouTube video title: ")
    max_comments = int(input("Max number of comments to scrape: "))
    scrape_youtube_comments(video_title, max_comments)


 Enter YouTube video title:  hp victus laptop
Max number of comments to scrape:  20



 Searching for: hp victus laptop
 Opening video: https://www.youtube.com/watch?v=jtDNtjULnOI&pp=ygUQaHAgdmljdHVzIGxhcHRvcA%3D%3D
 Loading comments...
 Saved 20 comments to 'youtube_comments.csv'
